# 1. 작업형 1

### 1. 주어진 데이터에서 ‘view’ 칼럼을 기준으로 내림차순으로 정렬하고 ‘views’ 칼럼 상위 10개 데이터를 ‘views’칼럼 상위 10번째 값으로 대체한 후 ‘age’ 칼럼이 80 이상인 데이터의 ‘views’ 칼럼의 평균값을 구하시오.(반올림하여 소수 둘째 자리까지 계산)

In [10]:
import pandas as pd
df = pd.read_csv('members.csv')
df_10th = df.sort_values('views', ascending = False)['views'].iloc[9]
df_sort = df.sort_values('views', ascending = False)[['views', 'age']] 
df_sort['views'].iloc[:10] = df_10th
print(round(df_sort[df_sort['age']>=80]['views'].mean(),2))

5674.04


### 2. 주어진 데이터셋에서 앞에서부터 순서대로 80% 데이터만 활용해 ‘f1’ 칼럼 결측치를 중앙값으로 채우기 전후의 표본 표준편차를 구하고, 두 표본 표준편차의 차이를 계산하시오.(두 표본 표준편차 차이는 절댓값으로 계산하고, 반올림하여 소수 둘째 자리까지 계산)

In [15]:
import pandas as pd
df = pd.read_csv('members.csv')
df_filter = df[:int(len(df) * 0.8)]
a = (df_filter['f1'].std())
b = (df_filter['f1'].fillna(df_filter['f1'].median()).std())
print(round(abs(a-b),2))

3.56


### 3. 주어진 데이터셋의 ‘age’ 칼럼의 이상치를 모두 더하시오. 이상치는 평균으로부터 ‘표준편차 * 1.5’를 벗어나는 영역을 이상치라고 판단한다. 

In [18]:
import pandas as pd
df = pd.read_csv('members.csv')
age_mean = df['age'].mean()
age_std = df['age'].std()
cond1 = df['age'] > age_mean + (1.5 * age_std)
cond2 = df['age'] < age_mean - (1.5 * age_std)
print(df[cond1 | cond2]['age'].sum())

473.5


# 작업형 2

### 제품 배송 시간에 맞춰 배송되지 않을 확률값을 예측하시오.

In [38]:
import pandas as pd
X_test = pd.read_csv('X_test.csv')
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
print(X_train.isnull().sum().sum(), X_test.isnull().sum().sum())

0 0


In [39]:
print(X_train.nunique())
print('----')
print(X_test.nunique())

ID                     8799
Warehouse_block           5
Mode_of_Shipment          3
Customer_care_calls       6
Customer_rating           5
Cost_of_the_Product     215
Prior_purchases           8
Product_importance        3
Gender                    2
Discount_offered         65
Weight_in_gms          3761
dtype: int64
----
ID                     2200
Warehouse_block           5
Mode_of_Shipment          3
Customer_care_calls       6
Customer_rating           5
Cost_of_the_Product     206
Prior_purchases           8
Product_importance        3
Gender                    2
Discount_offered         65
Weight_in_gms          1741
dtype: int64


In [40]:
object_col = X_train.select_dtypes(include = 'object').columns
for col in object_col:
    set_Xtrain = set(X_train[col])
    set_Xtest = set(X_test[col])
    same = (set_Xtrain) == (set_Xtest)
    if same : 
        print(col, "동일함")
    else:
        print(col, "동일하지 않음")

Warehouse_block 동일함
Mode_of_Shipment 동일함
Product_importance 동일함
Gender 동일함


In [33]:
from sklearn.preprocessing import LabelEncoder
le  = LabelEncoder()
cols = X_train.select_dtypes(include = 'object').columns
for col in cols:
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])
    
X_train = X_train.drop('ID', axis = 1)
X_test_id = X_test.pop('ID')

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train['Reached.on.Time_Y.N'],
                                                 test_size = 0.2, random_state = 0)
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
pred = rf.predict_proba(X_val)
from sklearn.metrics import roc_auc_score
score = roc_auc_score(y_val, pred[:,1])

pred = rf.predict_proba(X_test)
submit = pd.DataFrame({'ID' : X_test_id, 'Reached.on.Time_Y.N' : pred[:,1]})
print(submit.head())
submit.to_csv('result.csv', index= False)

      ID  Reached.on.Time_Y.N
0   8285                 0.47
1  10192                 0.47
2   8675                 0.40
3   5753                 0.41
4    448                 0.94


In [50]:
import pandas as pd
X_test = pd.read_csv('X_test.csv')
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')

from sklearn.preprocessing import LabelEncoder
le  = LabelEncoder()
cols = X_train.select_dtypes(include = 'object').columns
print(cols)
for col in cols:
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])
    
X_train = X_train.drop('ID', axis = 1)
X_test_id = X_test.pop('ID')

from sklearn.preprocessing import MinMaxScaler
cols = ['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases',
       'Discount_offered', 'Weight_in_gms']
scaler = MinMaxScaler()
X_train[cols] = scaler.fit_transform(X_train[cols])
X_test[cols] = scaler.transform(X_test[cols])

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train['Reached.on.Time_Y.N'],
                                                 test_size = 0.2, random_state = 0)
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
lg = lgb.LGBMClassifier(random_state = 0, verbose = -1)
lg.fit(X_train,y_train)
pred = lg.predict_proba(X_val)
print(roc_auc_score(y_val, pred[:,1]))

pred = rf.predict_proba(X_test)
submit = pd.DataFrame({'ID' : X_test_id, 'Reached.on.Time_Y.N' : pred[:,1]})
print(submit.head())
submit.to_csv('result.csv', index= False)

Index(['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender'], dtype='object')
0.7685566733735747
      ID  Reached.on.Time_Y.N
0   8285                 0.74
1  10192                 0.77
2   8675                 0.74
3   5753                 0.80
4    448                 0.79
